## Experiment 1

In [ ]:
# CUDA check
import torch
print(torch.cuda.is_available())

### Custom Tokenizer creation

In [ ]:
from transformers import AutoTokenizer
import pandas as pd
import os

# Load Base Tokenizer
base_model_path = "pythainlp/wangchanglm-7.5B-sft-enth" 
print("Loading base tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

#### custum words

In [ ]:
# read the old file
with open("custom_vocab_5w1h.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

# delete duplicated words and rearrange
unique_lines = sorted(set(line.strip() for line in lines if line.strip()))

print("\n----- words after preprocessing -----")
for line in unique_lines:
    print(line)

In [ ]:
# save file
with open("custom_vocab_5w1h.txt", "w", encoding="utf-8") as file:
    for line in unique_lines:
        file.write(line + "\n")

print("\nNew words saved")

In [ ]:
# load new words
with open("custom_vocab_5w1h.txt", "r", encoding="utf-8") as f:
    custom_tokens = [line.strip() for line in f.readlines() if line.strip()]

# add new words to tokenizer
num_added = tokenizer.add_tokens(custom_tokens)
print(f"✅ Added {num_added} new tokens.")

# save new tokenizer 
tokenizer.save_pretrained("custom_tokenizer_5w1h")

### Step0: Load dataset 

In [ ]:
import pandas as pd

df = pd.read_csv("Updated_Datasets.csv")
df.head(5)

### Step1: Load Custom Tokenizer

In [ ]:
from transformers import AutoTokenizer

# ✅ Load Custom Tokenizer
tokenizer_path = "./custom_tokenizer_5w1h" 
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

print("Custom Tokenizer loaded successfully.")
print("Vocab size:", len(tokenizer))


#### Tokenization Test

In [ ]:
def tokenize_and_check(text):
    if not isinstance(text, str):
        return None  # หรือ จะ return {"original_text": None, "tokens": [], "num_tokens": 0} ก็ได้
    encoding = tokenizer(text, add_special_tokens=False, return_tensors=None)
    token_ids = encoding['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    return {
        "original_text": text,
        "tokens": tokens,
        "num_tokens": len(tokens)
    }


In [ ]:
# Tokenize using Custom Tokenizer
def tokenize_and_check(text):
    if not isinstance(text, str):
        return None  # หรือ จะ return {"original_text": None, "tokens": [], "num_tokens": 0} ก็ได้
    encoding = tokenizer(text, add_special_tokens=False, return_tensors=None)
    token_ids = encoding['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    return {
        "original_text": text,
        "tokens": tokens,
        "num_tokens": len(tokens)
    }

tokenized_results_input1 = df['Input_Sec1'].apply(tokenize_and_check)

for idx, result in enumerate(tokenized_results_input1):
    print(f"\n==== Record {idx+1} ====")
    print(f"Original Text: {result['original_text'][:]}")  
    print(f"Number of Tokens: {result['num_tokens']}")
    print(f"Tokens: {result['tokens']}")
    if idx >= 2:
        break

In [ ]:
tokenized_results_input2 = df['Input_Sec2'].apply(tokenize_and_check)

for idx, result in enumerate(tokenized_results_input2):
    print(f"\n==== Record {idx+1} ====")
    print(f"Original Text: {result['original_text'][:]}")  
    print(f"Number of Tokens: {result['num_tokens']}")
    print(f"Tokens: {result['tokens']}")
    if idx >= 2:
        break

### Step 2: Preprocessing

In [ ]:
import re

def preprocess_text(text):
    # ตัดแบ่งประโยคเบื้องต้น (จุด, เว้นวรรค)
    sentences = re.split(r'(?<=[.])\s+', text)
    sentences = [s.strip() for s in sentences if s.strip()]
    
    # Boost Formality เล็กน้อย: แก้คำไม่เป็นทางการ (ตัวอย่าง)
    replacements = {
        "ช่วย": "กรุณา",
        "ดูแล": "ดำเนินการตรวจสอบ",
        "ดู": "ตรวจสอบ",
        "บอก": "แจ้ง",
        "ให้": "อนุเคราะห์",
    }
    boosted_sentences = []
    for sentence in sentences:
        for informal, formal in replacements.items():
            sentence = re.sub(rf'\b{informal}\b', formal, sentence)
        boosted_sentences.append(sentence)
    
    return " ".join(boosted_sentences)

df['Input_Sec1'] = df['Input_Sec1'].apply(preprocess_text)

for idx, row in df.iterrows():
    print(f"\n🧹 ตัวอย่าง Record {idx+1}")
    # print(f"Original Text:\n{row['combined_input'][:300]}...\n")
    print(f"Preprocessed Text:\n{row['Input_Sec1'][:300]}...")
    if idx >= 2:
        break

In [ ]:
df['Input_Sec2'] = df['Input_Sec2'].apply(preprocess_text)

for idx, row in df.iterrows():
    print(f"\n🧹 ตัวอย่าง Record {idx+1}")
    # print(f"Original Text:\n{row['combined_input'][:300]}...\n")
    print(f"Preprocessed Text:\n{row['Input_Sec2'][:300]}...")
    if idx >= 2:
        break

#### Filtering

In [ ]:
import re

# Expand abbreviation function
def expand_abbreviations(text, abbreviation_dict):
    for abbr, full_name in abbreviation_dict.items():
        pattern = r'\b' + re.escape(abbr) + r'\b'
        text = re.sub(pattern, f"{full_name} ({abbr})", text)
    return text

# Example abbreviation dictionary
abbreviation_dict = {
    "ก.ค.": "กรกฎาคม",
    "ก.พ.": "กุมภาพันธ์",
    "ก.ย.": "กันยายน",
    "กกท.ศทท.สส.ทหาร": "กองการโทรคมนาคม ศูนย์การโทรคมนาคมทหาร กรมการสื่อสารทหาร",
    "กกล.บก.สปท.": "กองกลาง กองบัญชาการ สถาบันวิชาการป้องกันประเทศ",
    "กขส.ยบ.ทหาร": "กองขนส่ง ยุทธบริการทหาร",
    "กตป.สปช.ทหาร": "กองติดตามและประเมินผล สำนักงานปลัดบัญชีทหาร",
    "กตส.สตป.": "กองตรวจสอบ สำนักงานตรวจสอบภายใน",
    "กทด.บก.สปท.": "กองทดสอบ กองบัญชาการ สถาบันวิชาการป้องกันประเทศ",
    "กทพ.กพ.ทหาร": "กองทัพพิเศษ กองกำลังพลทหาร",
    "กนผ.กร.ทหาร": "กองนโยบายและแผน กรมกิจการพลเรือนหาร",
    "กนผ.สนพ.กพ.ทหาร": "กองนโยบายและแผน สำนักงานนโยบายและแผน กองกำลังพลทหาร",
    "กพ.ทหาร": "กองกำลังพลทหาร",
    "กมศ.บก.สปท.": "กองมาตรฐานการศึกษา กองบัญชาการ สถาบันวิชาการป้องกันประเทศ",
    "กร.ทหาร": "กรมกิจการพลเรือนทหาร",
    "สบ.ทหาร": "กรมสารบรรณทหาร",
}


In [ ]:
# ขยายตัวย่อ
df['Input_Sec2'] = df['Input_Sec2'].apply(lambda x: expand_abbreviations(x, abbreviation_dict))
df['Input_Sec1'] = df['Input_Sec1'].apply(lambda x: expand_abbreviations(x, abbreviation_dict))

In [ ]:
def create_prompt_fewshot(input_sec2, input_sec1):
    prompt = f"""
    คุณคือผู้ช่วยสรุปหนังสือราชการ
    ให้อ่านข้อมูลที่กำหนด และสรุปเนื้อหาอย่างกระชับ ชัดเจน และเป็นภาษาทางการ
    ห้ามแต่งข้อมูลเพิ่มจากที่กำหนด
    ให้นำเสนอเป็นข้อความย่อความเท่านั้น ไม่ต้องจัดหมวดหมู่ ไม่ต้องตอบเป็น JSON

    ข้อความใหม่สำหรับสกัด:
    \"\"\"{input_sec2}\"\"\"

    เหตุผลเบื้องหลัง:
    \"\"\"{input_sec1}\"\"\"

    โปรดตอบเฉพาะสรุปข้อความย่อเป็นภาษาไทยเท่านั้น
    """
    return prompt


### Step 3: Experiment

#### Abstractive Mode: Ollama

In [ ]:
# Config สำหรับ Ollama
Ollama_API_URL = "http://localhost:11434/api/chat"
Ollama_Model_Name = "wangchanglm"

In [ ]:
import requests

def query_ollama_chat(prompt):
    payload = {
        "model": Ollama_Model_Name,
        "messages": [{"role": "user", "content": prompt}],
        "stream": False
    }
    try:
        response = requests.post(Ollama_API_URL, json=payload, timeout=120)
        response.raise_for_status()
        result = response.json()
        return result['message']['content']
    except Exception as e:
        print(f"⚠️ Error calling Ollama: {e}")
        return None


#### Extractive Mode

In [ ]:
from transformers import AutoModel

# Config WangchanBERTa (สำหรับ extractive)
model_name = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
def get_sentence_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].cpu().numpy()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def extract_sentences(title, input_sec1, content, top_k=3):
    if pd.isnull(title) or pd.isnull(content):
        return ""
    anchor_text = title + " " + input_sec1
    sentences = re.split(r'(?<=[.!?])\s+', content)
    sentences = [s.strip() for s in sentences if s.strip()]
    if not sentences:
        return ""
    anchor_emb = get_sentence_embedding(anchor_text)
    sentence_embs = [get_sentence_embedding(sent) for sent in sentences]
    sims = [cosine_similarity(anchor_emb, sent_emb)[0][0] for sent_emb in sentence_embs]
    top_indices = sorted(range(len(sims)), key=lambda i: sims[i], reverse=True)[:top_k]
    selected_sentences = [sentences[i] for i in top_indices]
    return ' '.join(selected_sentences)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

def rouge_l_score(ref, pred):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    return scorer.score(ref, pred)['rougeL'].fmeasure

def bleu_score(ref, pred):
    smoothie = SmoothingFunction().method4
    return sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothie)

#### Process

In [ ]:
# EXPERIMENT_MODE = ["extractive", "abstractive"]

In [ ]:
from tqdm import tqdm

extractive_outputs = []
abstractive_outputs = []
human_outputs = df['Output_Sec1'].tolist()

for idx, row in tqdm(df.iterrows(), total=len(df)):
    title = row['Title']
    input_sec1 = row['Input_Sec1']
    input_sec2 = row['Input_Sec2']

    # Extractive V2
    extracted = extract_sentences(title, input_sec1, input_sec2)
    extractive_outputs.append(extracted)

    # Abstractive V2
    input_sec2 = row['Input_Sec2']
    prompt = create_prompt_fewshot(input_sec2, input_sec1)
    # prompt = create_prompt_abstractive(input_sec2)
    summary = query_ollama_chat(prompt)
    if summary:
        abstractive_outputs.append(summary)
    else:
        abstractive_outputs.append("")

    # # Abstractive V2.1
    # summary = generate_summary_openthaigpt(input_sec2)
    # abstractive_outputs.append(summary)


### Step5: Evaluation

In [ ]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

similarity_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2') 

extractive_rouge = []
abstractive_rouge = []
extractive_bleu = []
abstractive_bleu = []

extractive_cosine = []
abstractive_cosine = []

for human, extr, abstr in tqdm(zip(human_outputs, extractive_outputs, abstractive_outputs), total=len(human_outputs)):
    # --- ROUGE-L ---
    extractive_rouge.append(rouge_l_score(human, extr))
    abstractive_rouge.append(rouge_l_score(human, abstr))

    # --- BLEU ---
    extractive_bleu.append(bleu_score(human, extr))
    abstractive_bleu.append(bleu_score(human, abstr))

    # --- Cosine Similarity ---
    emb_human = similarity_model.encode(human)
    emb_extr = similarity_model.encode(extr)
    emb_abstr = similarity_model.encode(abstr)

    cosine_extr = cosine_similarity([emb_human], [emb_extr])[0][0]
    cosine_abstr = cosine_similarity([emb_human], [emb_abstr])[0][0]

    extractive_cosine.append(cosine_extr)
    abstractive_cosine.append(cosine_abstr)


In [ ]:
df_result = pd.DataFrame({
    "Human_Reference": human_outputs,
    "Extractive_Output": extractive_outputs,
    "Abstractive_Output": abstractive_outputs,
    "Extractive_ROUGE_L": extractive_rouge,
    "Abstractive_ROUGE_L": abstractive_rouge,
    "Extractive_BLEU": extractive_bleu,
    "Abstractive_BLEU": abstractive_bleu,
    "Extractive_Cosine": extractive_cosine,
    "Abstractive_Cosine": abstractive_cosine
})


df_result.head()

In [ ]:
output_save_path = "Experiment1_Result.xlsx"
df_result.to_excel(output_save_path, index=False)

print(f"\n✅ All processing done. Result saved to {output_save_path}")

In [ ]:
# df.to_excel('output.xlsx', index=False, engine='openpyxl')

## Experiment 2

In [ ]:
# สร้างข้อความข้อ 2 (Fact) จากข้อ 1 แบบ Fixed Template

def create_fixed_fact(input_sec1):
    fixed_fact = ""
    return fixed_fact


In [ ]:
def create_prompt_for_section3(input_sec1, fixed_fact, with_feedback=False):
    base_prompt = f"""
<fact>
ข้อ ๑: {input_sec1}

ข้อ ๒: {fixed_fact}
</fact>

<instruction>
โปรดเขียนข้อเสนอหรือสั่งการ (ข้อ ๓) โดยปฏิบัติตามแนวทางต่อไปนี้:
- ลำดับเป็นข้อ ๆ: 2.1, 2.2, 2.3
- ห้ามเกิน 3 ข้อ
- แต่ละข้อเป็นประโยคคำสั่งสั้น ๆ (1–2 บรรทัด)
- ใช้ภาษาราชการที่สุภาพ และสอดคล้องกับข้อ ๑ และข้อ ๒
- ห้ามมีข้อความสรุปท้าย
</instruction>
"""

    if with_feedback:
        feedback_example = """
<example>
2.1 เรียนเชิญ ผบ.รร.ชท. หรือผู้แทนเข้าร่วมพิธีเปิดฯ ตามข้อ 1
2.2 กสน.ฯ จัดรถรับ-ส่ง เข้าร่วมพิธีเปิดฯ
2.3 ผธก.ฯ บันทึกลงระบบสารบรรณอิเล็กทรอนิกส์ (ECM) ให้หน่วยที่เกี่ยวข้องดำเนินการต่อไป
</example>
"""
        base_prompt = feedback_example.strip() + "\n" + base_prompt.strip()

    return base_prompt.strip()


In [ ]:
import numpy as np

def generate_best_section3(input_sec1, human_reference_section3, with_feedback=False):
    """
    Generate ข้อ 3 และคืนค่า: (Best Summary, Best Score)
    """
    # ✅ สร้างข้อ 2 จากข้อ 1
    fixed_fact = create_fixed_fact(input_sec1)

    # ✅ เตรียม Prompt
    prompt = create_prompt_for_section3(input_sec1, fixed_fact, with_feedback=with_feedback)

    generated_texts = []
    scores = []

    for _ in range(3):  # ยิง 3 ครั้ง
        gen_text = query_ollama_chat(prompt)
        if gen_text:
            generated_texts.append(gen_text)
            emb_human = similarity_model.encode(human_reference_section3)
            emb_generated = similarity_model.encode(gen_text)
            score = cosine_similarity([emb_human], [emb_generated])[0][0]
            scores.append(score)

    if not generated_texts:
        return "", 0.0

    best_idx = np.argmax(scores)
    best_text = generated_texts[best_idx]
    best_score = scores[best_idx]

    # (Optional) Postprocess
    # polished_text = rewrite_summary_wangchan(best_text)

    return best_text, best_score


In [ ]:
# ตัวอย่างสมมติ
input_sec1 = "ยบ.ทหาร (สนพ.ยบ.ทหาร) กําหนดพิธีเปิดการฝึกอบรมหลักสูตรนายทหารประทวน สายวิทยาการแพทย์ รุ่นที่ 3 ประจําปีงบประมาณ พ.ศ. 2568 ในวันพุธที่ 5 พ.ย. 67 เวลา 1300 ณ ห้องประชุม หทัยนเรศ ชั้น 2 สนพ.ยบ.ทหาร (บางซ่อน) โดยมี จก.ยบ.ทหาร เป็นประธานฯ การแต่งกาย เครื่องแบบปกติ คอพับแขนยาว (ทอ. อินทรธนูแข็ง) งดหมวก"

human_reference_section3 = "" \
"2.1 เรียนเชิญ ผบ.รร.ชท. หรือผู้แทนเข้าร่วมพิธีเปิดฯ ตามข้อ 1"
"2.2 กสน.ฯ จัดรถรับ-ส่ง เข้าร่วมพิธีเปิดฯ"
"2.3 ผธก.ฯ บันทึกลงระบบสารบรรณอิเล็กทรอนิกส์ (ECM) ให้หน่วยที่เกี่ยวข้องดําเนินการต่อไป" \
""

# ✅ Generate แบบมี Feedback Correction
section3_with_feedback, score_with_feedback = generate_best_section3(
    input_sec1, human_reference_section3, with_feedback=True
)

# ✅ Generate แบบไม่มี Feedback Correction
section3_without_feedback, score_without_feedback = generate_best_section3(
    input_sec1, human_reference_section3, with_feedback=False
)


# ✅ ดูผล
print("\n✅ Section 3 (With Feedback Correction):")
print(section3_with_feedback)
print("Cosine Similarity:", score_with_feedback)

print("\n✅ Section 3 (Without Feedback Correction):")
print(section3_without_feedback)
print("Cosine Similarity:", score_without_feedback)
